In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

In [2]:
# URL de la página principal
pagina_principal = "https://www.anuies.mx/anuies/instituciones-de-educacion-superior/"

In [3]:
# Función para obtener los enlaces a las subpáginas derivado de la pagina principal
def obtener_enlaces(pagina_principal):
    respuesta = requests.get(pagina_principal)
    soup = BeautifulSoup(respuesta.text, 'html.parser')
    
    
    # Encuentra todos los enlaces que apuntan a las subpáginas
    subpage_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'institucion.php' in href:
            full_url = f"https://www.anuies.mx{href}"
            subpage_links.append(full_url)
    
    return subpage_links

In [4]:
# Obtener los enlaces a las subpáginas
subpage_links = obtener_enlaces(pagina_principal)
#subpage_links

In [5]:
# Función para extraer la información de una subpágina
def get_university_info(subpage_url):
    response = requests.get(subpage_url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extrae la información que necesitas de la subpágina
    # Aquí debes ajustar los selectores según la estructura de la página
    
    institucion = soup.find('div', id='institucionesDivDespliegueTitulo').text.strip() if soup.find('div', id='institucionesDivDespliegueTitulo') else 'No disponible'
    
    # Buscar el div que contiene la lista de información general
    div_lista_general = soup.find('div', class_='institucionesDivGeneralesTexto')
    
    # Diccionario para almacenar los textos con nombres personalizados
    datos_generales = {
        'fecha_creacion': None,
        'niveles_acad': None,
        'niv_acad_1': None,
        'niv_acad_2': None,
        'niv_acad_3': None,
        'niv_acad_4': None,
        'niv_acad_5': None,
        'niv_acad_6': None,

    }
    
    if div_lista_general:
        elementos_li_general = div_lista_general.find_all('li')
        
        for i, li in enumerate(elementos_li_general):
            li_clon = BeautifulSoup(li.decode_contents(), 'html.parser')
            
            for span in li_clon.find_all('span'):
                span.decompose()  # Elimina el <span> y su contenido
            
            texto = li_clon.get_text(strip=True)
            
            if i == 0:
                datos_generales['fecha_creacion'] = texto
            elif i == 1:
                datos_generales['niveles_acad'] = texto
            elif i == 2:
                datos_generales['niv_acad_1'] = texto
            elif i == 3:
                datos_generales['niv_acad_2'] = texto
            elif i == 4:
                datos_generales['niv_acad_3'] = texto
            elif i == 5:
                datos_generales['niv_acad_4'] = texto
            elif i == 6:
                datos_generales['niv_acad_5'] = texto
            elif i == 7:
                datos_generales['niv_acad_6'] = texto
    
    # Buscar el div que contiene la lista de dirección
    div_lista_direccion = soup.find('div', class_='institucionesDivDireccion')
    
    # Diccionario para almacenar los textos con nombres personalizados
    datos_direccion = {
        'direccion_com': None,
        'direc_1': None,
        'direc_2': None,
        'direc_3': None,
        'direc_4': None,
        'direc_5': None,
        'direc_6': None,
        'telefono': None,
        'fax': None,
        'web': None,
        'email': None
    }
    
    if div_lista_direccion:
        elementos_li_direccion = div_lista_direccion.find_all('li')
        
        for i, li in enumerate(elementos_li_direccion):
            li_clon = BeautifulSoup(li.decode_contents(), 'html.parser')
            
            for span in li_clon.find_all('span'):
                span.decompose()  # Elimina el <span> y su contenido
            
            texto = li_clon.get_text(strip=True)
            
            if i == 0:
                datos_direccion['direccion_com'] = texto
            elif i == 1:
                datos_direccion['direc_1'] = texto
            elif i == 2:
                datos_direccion['direc_2'] = texto
            elif i == 3:
                datos_direccion['direc_3'] = texto
            elif i == 4:
                datos_direccion['direc_4'] = texto
            elif i == 5:
                datos_direccion['direc_5'] = texto
            elif i == 6:
                datos_direccion['direc_6'] = texto
            elif i == 7:
                datos_direccion['telefono'] = texto
            elif i == 8:
                datos_direccion['fax'] = texto
            elif i == 9:
                datos_direccion['web'] = texto
            elif i == 10:
                datos_direccion['email'] = texto
    
    # Combinar todos los datos en un solo diccionario
    datos_completos = {
        'institucion': institucion,
        **datos_generales,
        **datos_direccion
    }
    
    return datos_completos

In [6]:
# Recopilar la información de cada subpágina
universidades = []
for link in subpage_links:
    info = get_university_info(link)
    universidades.append(info)

# Crear un DataFrame con la información recopilada
df = pd.DataFrame(universidades)

In [7]:
df

,institucion,fecha_creacion,niveles_acad,niv_acad_1,niv_acad_2,niv_acad_3,niv_acad_4,niv_acad_5,niv_acad_6,direccion_com,direc_1,direc_2,direc_3,direc_4,direc_5,direc_6,telefono,fax,web,email
0,Instituto Tecnológico de Aguascalientes (ITA),18/09/1967,LicenciaturaPosgradoMaestríaDoctorado,Licenciatura,Posgrado,Maestría,Doctorado,None,None,"Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...","Av. Adolfo López Mateos N° 1801 Ote.\r, Col. F...",Aguascalientes,,Aguascalientes,Aguascalientes,20256,(449),(449),https://aguascalientes.tecnm.mx/,"dir_aguascalientes@tecnm.mx, direccion@aguasca..."
1,Universidad Autónoma de Aguascalientes (UAA),24/02/1974,BachilleratoLicenciaturaPosgradoEspecialidadMa...,Bachillerato,Licenciatura,Posgrado,Especialidad,Maestría,Doctorado,"Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...","Blvd. Av. Universidad N° 940\r, Edificio 1, Ci...",Aguascalientes,,Aguascalientes,Aguascalientes,20100,"(449) 910 74 10, 910 74 08",(449),https://www.uaa.mx/portal/,yesenia.pinzon@edu.uaa.mx
2,Universidad Politécnica de Aguascalientes (UPA),02/08/2002,LicenciaturaPosgradoMaestría,Licenciatura,Posgrado,Maestría,None,None,None,"Calle Paseo San Gerardo N° 207 \r, Fracc. San ...","Calle Paseo San Gerardo N° 207 \r, Fracc. San ...",Aguascalientes,,Aguascalientes,Aguascalientes,20342,(449),(449),https://www.upa.edu.mx/,otto.granados@upa.edu.mx
3,Universidad Tecnológica de Aguascalientes (UTAgs),11/08/1991,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,None,None,None,None,"Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...","Blvd. Juan Pablo II N° 1302\r, Fracc. Ex-Hacie...",Aguascalientes,,Aguascalientes,Aguascalientes,20200,(449),(449),https://www.utags.edu.mx/,armando.lopez@utags.edu.mx; luis.cardenas@utag...
4,Universidad Tecnológica de Calvillo (UTCalvillo),08/07/2013,Técnico SuperiorLicenciatura,Técnico Superior,Licenciatura,None,None,None,None,"Carr. al Tepetate 102\r, Col. El SalitreAguasc...","Carr. al Tepetate 102\r, Col. El Salitre",Aguascalientes,,Calvillo,Aguascalientes,20860,(495),(495),http://www.utcalvillo.edu.mx/,eduardo.blas@utcalvillo.edu.mx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,Instituto Tecnológico de Mérida (ITMérida),20/09/1961,LicenciaturaPosgradoMaestríaDoctorado,Licenciatura,Posgrado,Maestría,Doctorado,None,None,"Km. 5 Carr. Mérida-Progreso\r, Av. Tecnológico...","Km. 5 Carr. Mérida-Progreso\r, Av. Tecnológico...",Mérida,,Mérida,Yucatán,97118,(999) 964 50 01,(999),https://www.merida.tecnm.mx/,dir_merida@tecnm.mx; erika.pp@merida.tecnm.mx
240,Instituto Tecnológico Superior Progreso (ITSPr...,01/09/2000,LicenciaturaPosgradoMaestría,Licenciatura,Posgrado,Maestría,None,None,None,"Blvd. Víctor Manuel Cervera Pacheco s/n\r, x 6...","Blvd. Víctor Manuel Cervera Pacheco s/n\r, x 6...",Progreso,,Progreso,Yucatán,97320,(969),(969),http://www.progreso.tecnm.mx/,dir_dprogreso@tecnm.mx
241,Universidad Autónoma de Yucatán (UADY),01/01/1812,BachilleratoLicenciaturaPosgradoEspecialidadMa...,Bachillerato,Licenciatura,Posgrado,Especialidad,Maestría,Doctorado,"Calle 60 N° 491-A x 57 \r, Edif. Central Unive...","Calle 60 N° 491-A x 57 \r, Edif. Central Unive...",Mérida,,Mérida,Yucatán,97000,(999) 924 92 82,(999),https://www.uady.mx/,carlos.estrada@correo.uady.mx
242,Instituto Tecnológico de Zacatecas (ITZ),04/10/1976,Licenciatura,Licenciatura,None,None,None,None,None,"Carr. Panamericana\r, Entronque a Guadalajara ...","Carr. Panamericana\r, Entronque a Guadalajara ...",Zacatecas,,Zacatecas,Zacatecas,98000,(492),(492),https://zacatecas.tecnm.mx/,dir_ zacatecas@tecnm.mx


In [8]:
df.shape

(244, 18)

In [9]:
df.to_csv(os.path.join('..', 'datos', 'directorio_anuies.csv'))